In [1]:
import pandas as pd

# Helper functions

In [162]:
# create dfs for later merging

def make_df(filename):
    df = pd.read_csv(mypath + filename)
    df.name = filename.split('.')[0]
    val_name = df.name.split('_')[0]
    df.rename(columns={df.columns[0]:'country'}, inplace=True)
    #df.columns = map(lambda col: 
                    #'{}_{}'.format(str(col), name), df.columns)
    
    return df


In [29]:
# create list of filenames
from os import walk

mypath = 'data/original/'
def list_files(mypath):
    files = []
    [files.extend(filenames) for (dirpath, dirnames, filenames) in walk(mypath)]
    return files    

In [178]:
# reshape into long format for easier plotting

def reshape_for_plot(df, val_name):
    #df = df.reset_index().copy()
    #df = df.set_index(['region', 'sub-region', 'country'])
    name = df.name.split('_')[0]
    df = df.set_index(['country'])
    df = df.sort_index(level = 0)

    t = df.T
    t = t.unstack(level = 1)
    df = pd.DataFrame(t)
    
    #df = df.reset_index()
    df = df.rename(columns={'level_1':'year'+'_'+ name, 0: val_name})
    df.name = name

    return df

# Create wide DFs

In [30]:
files = list_files(mypath)

In [164]:
# verify
dfs = [make_df(filename) for filename in files]
len(dfs)

5

In [165]:
fert = dfs[0]
fert.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,...,6.04,5.82,5.60,5.38,5.17,4.98,4.80,4.64,4.48,4.33
1,Albania,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,4.60,...,1.65,1.65,1.67,1.69,1.70,1.71,1.71,1.71,1.71,1.71
2,Algeria,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,6.99,...,2.83,2.89,2.93,2.94,2.92,2.89,2.84,2.78,2.71,2.64
3,Angola,6.93,6.93,6.93,6.93,6.93,6.93,6.93,6.94,6.94,...,6.24,6.16,6.08,6.00,5.92,5.84,5.77,5.69,5.62,5.55
4,Antigua and Barbuda,5.00,5.00,4.99,4.99,4.99,4.98,4.98,4.97,4.97,...,2.15,2.13,2.12,2.10,2.09,2.08,2.06,2.05,2.04,2.03


# Create long DFs

In [73]:
#fert.reset_index(inplace=True)
fert_long = reshape_for_plot(fert, 'fertility')

In [74]:
fert_long.head()

,country,year,fertility
0,Afghanistan,1800,7.0
1,Afghanistan,1801,7.0
2,Afghanistan,1802,7.0
3,Afghanistan,1803,7.0
4,Afghanistan,1804,7.0


In [75]:
files

['children_per_woman_total_fertility.csv',
 'hdi_human_development_index.csv',
 'income_per_person_gdppercapita_ppp_inflation_adjusted.csv',
 'roads_paved_percent_of_total_roads.csv',
 'surviving_kids_per_woman.csv']

In [89]:
def get_val_names(dfs):
    names = [df.name.split('_')[0] for df in dfs]
    return names

In [90]:
val_names = get_val_names(dfs)

In [91]:
val_names

['children', 'hdi', 'income', 'roads', 'surviving']

In [94]:
dfnames = [df.name for df in dfs]
dfnames

['children_per_woman_total_fertility',
 'hdi_human_development_index',
 'income_per_person_gdppercapita_ppp_inflation_adjusted',
 'roads_paved_percent_of_total_roads',
 'surviving_kids_per_woman']

In [104]:
test = zip(dfnames, val_names)

In [108]:
test[0]

"('children_per_woman_total_fertility', 'children')"

In [99]:
for pair in test:
    print pair

('children_per_woman_total_fertility', 'children')
('hdi_human_development_index', 'hdi')
('income_per_person_gdppercapita_ppp_inflation_adjusted', 'income')
('roads_paved_percent_of_total_roads', 'roads')
('surviving_kids_per_woman', 'surviving')


In [174]:
long_dfs = [reshape_for_plot(dfs[i], val_names[i]) for i, _ in enumerate(dfs)]
len(long_dfs)

5

In [175]:
long_dfs[4].head()

,country,year_surviving,surviving
0,Afghanistan,1760,NaN
1,Afghanistan,1761,NaN
2,Afghanistan,1762,NaN
3,Afghanistan,1763,NaN
4,Afghanistan,1764,NaN


In [120]:
co2_regions = pd.read_pickle('data/co2_regions.pkl')
co2_regions.reset_index(inplace=True)

In [121]:
co2_long = reshape_for_plot(co2_regions, 'CO')

# Join long DFs

In [177]:
def join_dfs(ldf, rdf):
    rsuffix = '_' + rdf.name.split('_')[0]
    return ldf.join(rdf, rsuffix=rsuffix, how='outer')

final_df = reduce(join_dfs, long_dfs) #that's the magic
final_df.head()

,country,year_children,children,country_hdi,year_hdi,hdi,country_income,year_income,income,country_roads,year_roads,roads,country_surviving,year_surviving,surviving
0,Afghanistan,1800,7.0,Afghanistan,1990,0.295,Afghanistan,1800,603.0,Afghanistan,1990,NaN,Afghanistan,1760,NaN
1,Afghanistan,1801,7.0,Afghanistan,1991,0.300,Afghanistan,1801,603.0,Afghanistan,1991,NaN,Afghanistan,1761,NaN
2,Afghanistan,1802,7.0,Afghanistan,1992,0.309,Afghanistan,1802,603.0,Afghanistan,1992,NaN,Afghanistan,1762,NaN
3,Afghanistan,1803,7.0,Afghanistan,1993,0.305,Afghanistan,1803,603.0,Afghanistan,1993,NaN,Afghanistan,1763,NaN
4,Afghanistan,1804,7.0,Afghanistan,1994,0.300,Afghanistan,1804,603.0,Afghanistan,1994,NaN,Afghanistan,1764,NaN
